In [1]:
# 참고자료: https://github.com/davidsandberg/facenet
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import math

# 주성분분석(Principal Analysis)을 하기위한 라이브러리
from sklearn.decomposition import PCA

# 파일 경로 확인용
import os
import glob

# .py 파일 가져오기
# import (폴더이름.)파일이름
import src.facenet as facenet
import src.align.detect_face as detect_face

In [2]:
### 0. 내얼굴들의 임베딩을 미리 계산(실습 1의 embeded_me)
# jupyter notebook 페이지에 그르기
%matplotlib inline

In [3]:
# 학습되어있는 모델을 로드하는 함수
#    입력: protobuf 파일(모델이 정의되어 있음)
#    리턴:
#        - single_image: 영상을 입력하는 placeholder (session.run feed_dict에 사용)
#        - embeddings: 네트워크의 출력 값, 512차원의 추출된 얼굴 특징 벡터

def load_model(pb_path, image_size=(160,160)):
    tf.reset_default_graph()
    
    single_image = tf.placeholder(tf.int32, (None,None,3))
    float_image = tf.cast(single_image, tf.float32)
    float_image = float_image / 255
    batch_image = tf.expand_dims(float_image, 0)
    resized_image = tf.image.resize(batch_image, image_size)
    
    phase_train = tf.placeholder_with_default(False, shape=[])
    
    input_map = {'image_batch':resized_image, 'phase_train':phase_train}
    model = facenet.load_model(pb_path, input_map)
    
    embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
    
    return single_image, embeddings

In [4]:
# 영상 경로를 입력박아 로드하고, return해주는 함수
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    return image

# 두 벡터간의 거리를 계산하는 함수
def calc_distance(embedding1, embedding2):
    # Euclidian distance
    diff = np.subtract(embedding1, embedding2)
    dist = np.sum(np.square(diff),0)
    return dist

In [5]:
## 함수정의
# 단일 RGB영상과 pre-trained 네트워크(P/R/0 net)를 입력받아
#   1. face detection
#   2. crop (margin 추가)
#   3. resize 하고,
#  검출된 얼굴 영상들의 리스트를 반환하는 함수

def crop_faces(image, pnet, rnet, onet):
    # face detection 관현 파라미터
    
    minsize = 20 #minimum size of face
    threshold = [0.6, 0.7, 0.7] # three step's threshold
    factor = 0.709 # scale factor
    
    # crop 관련 파라미터
    margin = 44 # 상하좌우 여백
    image_size = 160 # resize(scaling) 크기
    
    h,w,_ = np.shape(image)
    
    # 얼굴 검출
    bounding_boxes, points = detect_face.detect_face(image, minsize, pnet, rnet, onet, threshold, factor)
    # bounding_boxes: 검출된 사각형 영역,
    #                 [x1,y1,x2,y2,확률]로 이루어진 리스트
    # points: 검출된 얼굴의 주요 landmark,
    #         [x1,x2,x3,x4,x5,y1,y2,y3,y4,y5]로 이루어진 리스트
    
    faces = []
    for box in bounding_boxes:
        box = np.int32(box)
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(box[0]-margin/2, 0)
        bb[1] = np.maximum(box[1]-margin/2, 0)
        bb[2] = np.minimum(box[2]+margin/2, w)
        bb[3] = np.minimum(box[3]+margin/2, h)
        cropped = image[bb[1]:bb[3],bb[0]:bb[2],:]
        scaled = cv2.resize(cropped, (image_size, image_size), interpolation=cv2.INTER_LINEAR)
        
        faces.append(scaled)
    
    return faces, bounding_boxes

In [6]:
## 모델 및 세션 선언
tf.reset_default_graph()
single_image, embeddings = load_model("./models/20180402-114759.pb")

sess = tf.Session()

## 학습되어 있는 detection 모델 로드
pnet, rnet, onet = detect_face.create_mtcnn(sess, None)

Model filename: ./models/20180402-114759.pb
Instructions for updating:
Use tf.gfile.GFile.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [7]:
## 내 얼굴의 특징벡터(embedding)추출

path_me = glob.glob("./project1/data/faces/me/*")

embed_me = []

for path in path_me:
    img = load_image(path)
    result = sess.run(embeddings, feed_dict={single_image:img})
    result = result[0]
    embed_me.append(result)
    
embed_me = np.array(embed_me)
print(embed_me.shape)

(33, 512)


In [8]:
### 1. 동영상 로드
cap = cv2.VideoCapture("./project1/data/faces/video/people.mp4")

while(cap.isOpened()):
    ret, frame = cap.read()
    
    frame = cv2.resize(frame,(400,225))
    
    # 2. 프레임 별로 얼굴 검출
    faces, bounding_boxes = crop_faces(frame, pnet, rnet, onet)
    number_f = len(faces)
    
    squared_img = frame.copy()
    for box in bounding_boxes:
        box = np.int32(box)
        p1 = (box[0], box[1])
        p2 = (box[2], box[3])
        # if 내얼굴 == 파란색
        # else == 빨간색
        cv2.rectangle(squared_img, p1, p2, color=(255,0,0))

    cv2.imshow('frame', frame)
    cv2.imshow('detected', squared_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()